In [1]:
import torch
import torchvision
import torch.nn as nn
torch.set_printoptions(profile="full") # or 'default'

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
import pandas as pd

import cupy as cp
cp.cuda.Device(1).use()

import matplotlib.pyplot as plt
%matplotlib inline

import sys
import numpy as np
np.set_printoptions(suppress=True, linewidth=np.nan, precision=2, threshold=sys.maxsize)

In [6]:
USE_GPU = True
device = torch.device("cuda" if USE_GPU else "cpu")
trainset = torchvision.datasets.MNIST('./data', train=True, transform=torchvision.transforms.ToTensor())
testset = torchvision.datasets.MNIST('./data', train=False, transform=torchvision.transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=False, num_workers=6)
        

def vis_fc(act_list, layer_idx):
    fc_act = act_list[layer_idx][0].data.cpu().numpy()
    assert(len(fc_act)%20 == 0)
    plt.imshow(fc_act.reshape(10,len(fc_act)//10), cmap='gray')
    plt.suptitle('fc layer result, shape (1,500) ', fontsize=20)
    plt.savefig("fc.png")
    plt.show()
    

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.functional.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28*1, 256, bias=False)
        self.fc2 = nn.Linear(256, 256, bias=False)
        self.fc3 = nn.Linear(256, 10, bias=False)

    def forward(self, x):
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        x = nn.functional.relu(x)
        x = self.fc3(x)
        return nn.functional.log_softmax(x, dim=1)

model = Net()
model.load_state_dict(torch.load("./model/mlp_model_1layer_256"))
model = model.to(device)
model.eval()

Net(
  (fc1): Linear(in_features=784, out_features=256, bias=False)
  (fc2): Linear(in_features=256, out_features=256, bias=False)
  (fc3): Linear(in_features=256, out_features=10, bias=False)
)

Prepare Data for MNIST dataset
---

In [7]:
mnist_train_data = []
mnist_train_label = []
mnist_test_data = []
mnist_test_label = []
    
for image, label in trainset:    
    img = np.array(image.view(-1))
    mnist_train_data.append(img)
    mnist_train_label.append(label)
    
for image, label in testset:    
    img = np.array(image.view(-1))
    mnist_test_data.append(img)
    mnist_test_label.append(label)
    
mnist_train_data = np.vstack(mnist_train_data)
mnist_train_label = np.vstack(mnist_train_label).reshape(-1)
mnist_test_data = np.vstack(mnist_test_data)
mnist_test_label = np.vstack(mnist_test_label).reshape(-1)

print("train", mnist_train_data.shape, mnist_train_label.shape)
print("test", mnist_test_data.shape, mnist_test_label.shape)

train (60000, 784) (60000,)
test (10000, 784) (10000,)


PCA Analysis
---

In [29]:
asdf = np.array([[-1,2],[3,4]])
print(asdf)
np.linalg.eigh(asdf)

[[-1  2]
 [ 3  4]]


(array([-2.41,  5.41]), array([[-0.91,  0.42],
        [ 0.42,  0.91]]))

In [30]:
asdf_inv = np.invert(asdf)

In [31]:
asdf_inv

array([[ 0, -3],
       [-4, -5]])

In [32]:
np.linalg.eigh(asdf_inv)

(array([-7.22,  2.22]), array([[ 0.48, -0.87],
        [ 0.87,  0.48]]))

In [19]:
pca = PCA()
pca.fit_transform(asdf.T)

array([[-1.41,  0.  ],
       [ 0.  ,  0.  ],
       [ 1.41,  0.  ]])

In [20]:
pca.components_

array([[ 0.71,  0.71],
       [ 0.71, -0.71]])

In [7]:
pca.singular_values_

array([2., 0.])

Dataset after PCA
---

PCA Testing Acceleration
---

In [22]:
logisticRegr = LogisticRegression()
%time logisticRegr.fit(mnist_train_data, mnist_train_label)

CPU times: user 1min 5s, sys: 574 ms, total: 1min 6s
Wall time: 8.56 s


/home/jjiangan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [26]:
%time logisticRegr.score(mnist_test_data, mnist_test_label)

CPU times: user 90.2 ms, sys: 28.2 ms, total: 118 ms
Wall time: 60.2 ms


0.9256

In [31]:
# Initialize the Scaler class object
pca = PCA(n_components=0.9)
# Perform PCA
data_reduced = pca.fit_transform(mnist_test_data)
print(data_reduced.shape)

(10000, 84)
